<a href="https://colab.research.google.com/github/smekur/HSE_python_course_KS/blob/master/%D0%A1%D0%B1%D0%BE%D1%80%20%D1%81%D0%B2%D0%BE%D0%B5%D0%B3%D0%BE%20%D0%BA%D0%BE%D1%80%D0%BF%D1%83%D1%81%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этой тетрадке мы поговорим о способах собрать свой датасет для исследований: откуда брать данные, как их собирать и как хранить.

Достаточно часто, чтобы собрать данные, нужно что-то выкачать с сайта

Существует несколько библиотек(модулей) для работы с веб-страничками, сегодня мы будем использовать **requests** для доступа к веб-страничкам и **Beautiful Soup** для работы с содержимым html-документов

In [0]:
! pip3 install requests # импортируем модуль requests

In [0]:
# импортируем модуль beautifulsoup, самая последняя версия - четвертая

! pip3 install beautifulsoup4 

In [0]:
# импортируем модули в тетрадку

import requests as rq

from bs4 import BeautifulSoup

## работаем с веб-страничками

Мы установили и импортировали модули, теперь можем приступать к работе

шаг 1. 

Создадим переменную ```url``` и сохраним в нее адрес какой-нибудь html-страницы

например, [сайта CNN](http://lite.cnn.io/en)

обратите внимание, что адрес прописываем в кавычках, как строку

In [0]:
url = 'https://www.theguardian.com/uk-news'

В модуле requests есть метод request.get(), который сохраняет ответ сервера на наш реквест. Мы применим его к переменной url, куда сохранен путь к странице. 
Сохраним результат в переменную page

In [0]:
page = rq.get(url)

In [0]:
page # посмотрим на код ответа, если 200, все хорошо

<Response [200]>

In [0]:
? page

In [0]:
# статус-код можно вызвать, написав .status_code после page (без скобок, тк это атрибут, а не метод)

page.status_code

200

код 200 сообщает, что страница загружена успешно 
*(коды, начинающиеся с 2, обычно указывают на успешное выполнение операции, а коды, начинающиеся с 4 или 5, сообщают об ошибке)*

Узнать больше о кодах состояния HTTP  можно [по этой ссылке.](https://www.w3.org/Protocols/HTTP/1.1/draft-ietf-http-v11-spec-01#Status-Codes)

Следующим шагом нужно получить доступ к текстовому содержимому веб-файлов.

Здесь нам поможет page.text *(или page.content, чтобы получить значение в байтах)*

In [0]:
page.text

'\n<!DOCTYPE html>\n<html id="js-context" class="js-off is-not-modern id--signed-out" lang="en" data-page-path="/uk-news">\n<head>\n<!--\n     __        __                      _     _      _\n     \\ \\      / /__    __ _ _ __ ___  | |__ (_)_ __(_)_ __   __ _\n      \\ \\ /\\ / / _ \\  / _` | \'__/ _ \\ | \'_ \\| | \'__| | \'_ \\ / _` |\n       \\ V  V /  __/ | (_| | | |  __/ | | | | | |  | | | | | (_| |\n        \\_/\\_/ \\___|  \\__,_|_|  \\___| |_| |_|_|_|  |_|_| |_|\\__, |\n                                                            |___/\n    Ever thought about joining us?\n    https://workforus.theguardian.com/careers/digital-development/\n     --->\n<title>Latest UK news | The Guardian</title>\n<meta charset="utf-8">\n<meta name="description" content="Latest news, breaking news and current affairs coverage from across the UK from theguardian.com"/>\n<meta http-equiv="X-UA-Compatible" content="IE=Edge"/>\n<meta name="format-detection" content="telephone=no"/>\n<meta name="Handhe

## работаем с текстом на страничке

Мы получили текст страницы (со всеми html-тегами), однако его неудобно прочитать в таком виде. 

Здесь нам понадобится Beautiful Soup, модуль для html-парсинга: он сделает текст веб-страницы, извлеченный с помощью Requests, более читаемым, потому что создает дерево синтаксического разбора из проанализированных HTML (или XML) документов.

In [0]:
soup = BeautifulSoup(page.text, 'html.parser') #сохраним результат в переменную soup

In [0]:
print(soup.prettify()) # показывает нашу страницу в красивом виде

<!DOCTYPE html>
<html class="js-off is-not-modern id--signed-out" data-page-path="/uk-news" id="js-context" lang="en">
 <head>
  <!--
     __        __                      _     _      _
     \ \      / /__    __ _ _ __ ___  | |__ (_)_ __(_)_ __   __ _
      \ \ /\ / / _ \  / _` | '__/ _ \ | '_ \| | '__| | '_ \ / _` |
       \ V  V /  __/ | (_| | | |  __/ | | | | | |  | | | | | (_| |
        \_/\_/ \___|  \__,_|_|  \___| |_| |_|_|_|  |_|_| |_|\__, |
                                                            |___/
    Ever thought about joining us?
    https://workforus.theguardian.com/careers/digital-development/
     --->
  <title>
   Latest UK news | The Guardian
  </title>
  <meta charset="utf-8"/>
  <meta content="Latest news, breaking news and current affairs coverage from across the UK from theguardian.com" name="description">
   <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
   <meta content="telephone=no" name="format-detection"/>
   <meta content="True" name="Handhel

##работаем с тегами в тексте: 

предыдущие шаги позволили привести веб-страничку к виду, где содержание каждого тега написано с новой строки. 

Некоторые теги полезны для конкретной задачи (там текст), некоторые - не очень (например, мета-данные,картинки и тд)

Извлечь один тег со страницы можно с помощью метода find_all(). Он похож на метод регулярок, с которым мы работали: он вернет все экземпляры данного тега в документе. Нужно прописать в скобках метода нужный тег. 

Текст содержится в разделах с тегом \<p>

In [0]:
soup.find_all('p') 

# попробуйте теги head, body, title, div 

[<p>The Guardian's Kate Proctor dispels some Christmas election myths.</p>,
 <p>Rhodes was best known for presenting hit TV shows such as MasterChef and Rhodes Around Britain</p>,
 <p>The SNP leader said: ‘A vote for the SNP is a vote to escape Brexit’ as she launched her party’s election manifesto</p>,
 <p>The Labour leader said the dossier leaves Boris Johnson’s denials on post-Brexit US trade deal ‘in absolute tatters’</p>,
 <p>Jeremy Corbyn has responded to the <a href="https://www.theguardian.com/politics/2019/nov/25/labour-has-let-poison-of-antisemitism-take-root-says-chief-rabbi">chief rabbi</a> by saying a Labour government will not tolerate antisemitism in any form</p>,
 <p>The prime minister has ruled out a second vote on Brexit or Scottish independence in the event of a hung parliament</p>,
 <p>In the third part of their election series, John Harris and John Domokos explore Southend's homelessness crisis</p>]

Результат метода .find_all() хранится в виде списка. 

Можно итерироваться элементам списка (например, третий элемент из всех с тегами \<p>)

In [0]:
soup.find_all('p')[3]

<p>The Labour leader said the dossier leaves Boris Johnson’s denials on post-Brexit US trade deal ‘in absolute tatters’</p>

метод .get_text() позволит вытащить текст из элемента:

In [0]:
links = []
#souplist = soup.findall("p")[h]
#for elem in souplist:
 #   links.append(elem[href])

In [0]:
for i in soup.find_all('class="fc-sublink__link" data-link-name="article"'):
    links.append(i)
print(links)

[<a class="u-h skip" data-link-name="skip : main content" href="#maincontent">Skip to main content</a>, <a class="new-header__logo" data-link-name="nav2 : logo" href="https://www.theguardian.com/us">
<span class="u-h">The Guardian - Back to home</span>
<span class="inline-the-guardian-logo inline-logo ">
<svg class="inline-the-guardian-logo__svg inline-logo__svg" viewbox="0 0 297 95">
<path d="M66.8 50.7l5-2.6V8.4H68l-9.3 12.4h-1L58.2 7h40.5l.6 13.8h-1.1L89 8.4h-3.9V48l5 2.7V52H66.9v-1.3zm37-1.8V5L100 3.5v-.9L114.2.1h1.5v20.8l.3-.4a19 19 0 0 1 12.2-4.5c6.2 0 9 3.5 9 10v23l3.3 1.7V52H122v-1.3l3.4-1.8V26c0-3.6-1.6-5-4.6-5a7.8 7.8 0 0 0-4.9 1.6V49l3.3 1.8V52h-18.5v-1.2zm48.4-13.4c.4 7.2 3.6 12.8 11.4 12.8 3.7 0 6.3-1.7 8.8-3v1.5a17.4 17.4 0 0 1-13.6 6.2c-12 0-18-6.6-18-18.1 0-11.3 6.6-18.3 17.4-18.3 10.2 0 15.5 5 15.5 18.4v.4zm-.2-1.7l10.5-.7c0-9-1.5-15-4.6-15-3.3 0-5.9 7-5.9 15.6M0 69.6c0-19.1 12.7-26 26.8-26 6 0 11.6 1 14.8 2.3l.3 13.4h-1.4l-8.3-13a12.2 12.2 0 0 0-5.2-.8c-7.5 0-11.3 8.7

## полезные ссылки

[документация requests и быстрый гайд](https://requests.readthedocs.io/en/master/user/quickstart/)


[документация Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

[text-only](https://sjmulder.nl/en/textonly.html) веб-сайты, чтобы легко начать парсить



[здесь](https://www.york.ac.uk/teaching/cws/wws/webpage1.html) можно почитать про структуру html подробнее


а [здесь](https://www.w3schools.com/html/html_examples.asp) еще и потренироваться в режиме онлайн

## try-except

В блоке try мы выполняем инструкцию, которая может породить исключение, а в блоке except мы перехватываем их.



In [0]:
def divider():
    x = float(input('Введите число:'))
    try:
        res = 1/x
        print('получается', res)
    except ZeroDivisionError :
        print("я не умею делить на 0 :( ")
        #res = 1/x
    #print('получается', res)
    

In [0]:
divider()

Введите число:1
получается 1.0


**типы эксепшонов:**

все можно почитать [тут](https://docs.python.org/3/tutorial/errors.html):

BaseException - базовое исключение, от которого берут начало все остальные.

KeyboardInterrupt - порождается при прерывании программы пользователем (обычно сочетанием клавиш Ctrl+C).

StopIteration - порождается встроенной функцией next, если в итераторе больше нет элементов.

ArithmeticError - арифметическая ошибка.

* FloatingPointError - порождается при неудачном выполнении операции с плавающей запятой. На практике встречается нечасто.

* OverflowError - возникает, когда результат арифметической операции слишком велик для представления. Не появляется при обычной работе с целыми числами (так как python поддерживает длинные числа), но может возникать в некоторых других случаях.

* ZeroDivisionError - деление на ноль.

AttributeError - объект не имеет данного атрибута (значения или метода).

EOFError - функция наткнулась на конец файла и не смогла прочитать то, что хотела.

ImportError - не удалось импортирование модуля или его атрибута.

LookupError - некорректный индекс или ключ.

IndexError - индекс не входит в диапазон элементов.

KeyError - несуществующий ключ (в словаре, множестве или другом объекте).

MemoryError - недостаточно памяти.

NameError - не найдено переменной с таким именем.

UnboundLocalError - сделана ссылка на локальную переменную в функции, но переменная не определена ранее.

OSError - ошибка, связанная с системой.

FileExistsError - попытка создания файла или директории, которая уже существует.

FileNotFoundError - файл или директория не существует.

InterruptedError - системный вызов прерван входящим сигналом.

IsADirectoryError - ожидался файл, но это директория.

NotADirectoryError - ожидалась директория, но это файл.

PermissionError - не хватает прав доступа.

TimeoutError - закончилось время ожидания.

RuntimeError - возникает, когда исключение не попадает ни под одну из других категорий.

SyntaxError - синтаксическая ошибка.

IndentationError - неправильные отступы.

TabError - смешивание в отступах табуляции и пробелов.

SystemError - внутренняя ошибка.

TypeError - операция применена к объекту несоответствующего типа.

ValueError - функция получает аргумент правильного типа, но некорректного значения.

UnicodeError - ошибка, связанная с кодированием / раскодированием unicode в строках.

UnicodeEncodeError - исключение, связанное с кодированием unicode.

UnicodeDecodeError - исключение, связанное с декодированием unicode.

UnicodeTranslateError - исключение, связанное с переводом unicode.

Warning - предупреждение.